<a href="https://colab.research.google.com/github/JoeXia77/Tools/blob/main/%E6%A0%B9%E6%8D%AE%E8%AF%BE%E4%BB%B6ppt%E7%94%9F%E6%88%90%E8%AE%B2%E4%B9%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
## 使用方法：
## 需要：
## 1. CHATGLM_API_KEY， 将之放置于google drive下的该位置：/content/drive/My Drive/API_keys/CHATGLM_API_KEY.txt
## 2. 在google drive中建立如下文件夹，以存放过程文件，以及最终的结果（在final_output里）
## 3. 课件文件为.pptx格式
## need to create following folders in your google drive first,
ppt_folder_path = "/content/drive/My Drive/datasets/pptx"
ppt_extracted_content_folder_path = "/content/drive/My Drive/datasets/pptx_to_txt"
summary_outline_folder_path = "/content/drive/My Drive/datasets/summary_outline"
final_output_folder_path = "/content/drive/My Drive/datasets/final_output"


In [132]:
import os
from google.colab import drive
## set up API key, could use: os.environ.get('CHATGLM_API_KEY') to retrive it
drive.mount('/content/drive')
api_key_path = '/content/drive/My Drive/API_keys/CHATGLM_API_KEY.txt'
def set_api_key_env(file_path):
    with open(file_path, 'r') as file:
        os.environ['CHATGLM_API_KEY'] = file.read().strip()
set_api_key_env(api_key_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
!pip install --upgrade zhipuai
!pip install python-pptx

In [134]:
import json
from pptx import Presentation
import json
from zhipuai import ZhipuAI

## Extract text in pptx file

In [135]:
def read_pptx(file_path):
    prs = Presentation(file_path)
    slides_content = []
    for i, slide in enumerate(prs.slides, start=1):
        slide_text = []
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                slide_text.append(shape.text)
        content_string = ' '.join(slide_text)
        slides_content.append((f'pptx第{i}页', content_string))
    return slides_content

def process_pptx_files(source_folder_path, target_folder_path):
    # Ensure target directory exists, create if not
    os.makedirs(target_folder_path, exist_ok=True)
    for file_name in os.listdir(source_folder_path):
        if file_name.endswith('.pptx'):
            file_path = os.path.join(source_folder_path, file_name)
            slides_content = read_pptx(file_path)
            text_file_name = os.path.splitext(file_name)[0] + '.txt'
            # Save the content with non-escaped Unicode characters
            with open(os.path.join(target_folder_path, text_file_name), 'w', encoding='utf-8') as txt_file:
                json.dump(slides_content, txt_file, ensure_ascii=False)

source_folder_path = '/content/drive/My Drive/datasets/pptx'
target_folder_path = '/content/drive/My Drive/datasets/pptx_to_txt'
process_pptx_files(source_folder_path, target_folder_path)


In [136]:
# Class definition
class Llm_processor:
    def __init__(self):
        ## define model here
        self.model_name = "GLM-4-Air"
        self.client = ZhipuAI(api_key=os.environ.get('CHATGLM_API_KEY'))

    def process(self, chat_history):
        ## chat history should be a list of dict, like this:
        ##  [{"role": "user", "content": "text"}, {"role": "assistant", "content": "text"}, {"role": "user", "content": f"{text}"}]
        result = self.client.chat.completions.create(
            model=self.model_name,
            messages=chat_history,
        )
        return result

processor = Llm_processor()


In [137]:
def load_text_files_to_list(folder_path):
    all_data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            full_path = os.path.join(folder_path, file_name)
            with open(full_path, 'r', encoding='utf-8') as txt_file:
                # Load the content from the file
                content = json.load(txt_file)
                # Convert tuples back to lists if necessary
                content = [list(item) if isinstance(item, tuple) else item for item in content]
                all_data.append([file_name, content])
    return all_data

source_folder_path = '/content/drive/My Drive/datasets/pptx_to_txt'
pptx_data = load_text_files_to_list(source_folder_path)


In [138]:
def summarize(content):
    system_role = "你是一名专业的教师，你会尽全力帮助用户撰写高质量的教学资料。"
    user_message_part1 = "请总结下文课件中的重要内容: "
    user_message_part2 = content
    user_message = user_message_part1 + user_message_part2

    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_message}
    ]
    response = processor.process(messages)
    result = response.choices[0].message.content
    return result

In [139]:
def generate_outline(content):
    system_role = "你是一名专业的教师，你会尽全力帮助用户撰写高质量的教学资料，并将撰写的信息以清晰的 JSON格式呈现"
    user_message_part1 = '请参考如下课件的内容\n /// '
    user_message_part2 = content
    user_message_part3 = '///\n 为授课教师写一份安排合理的教学大纲，请确保用json格式输出你的结果且保证输出中不含有非json格式的内容，具体格式如下：[["大纲的第1部分"，该部分涉及到的pptx的页码如涉及到2，3，5页则写为：[2,3,5]], ["大纲的第2部分", ...], ...]'

    user_message = user_message_part1 + user_message_part2 + user_message_part3

    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_message}
    ]
    response = processor.process(messages)
    result = response.choices[0].message.content

    ## format json
    start_index = result.find('[')
    end_index = result.rfind(']')

    # Extract the JSON content if both '[' and ']' are found
    if start_index != -1 and end_index != -1:
        result = result[start_index:end_index + 1]

    return result


In [140]:
pptx_data

[['第二章 《碳交易市场理论基础》--许光清.txt',
  [['pptx第1页', ' 第二章 碳交易市场理论基础   高等学校碳金融“十四五”规划教材'],
   ['pptx第2页', ' 外部性理论 公共物品理论 全球碳减排的囚徒困境 目录CONTENTS 科斯理论'],
   ['pptx第3页', '外部性理论 01'],
   ['pptx第4页', '一、市场失灵'],
   ['pptx第5页',
    '完全竞争市场经济在一系列理想化假定条件下，可以导致整个经济达到一般均衡，使资源配置达到帕累托最优状态。但是完全竞争所依赖的假定条件很多是非常严格的。因此，完全竞争市场和其他一系列理想化假定条件并不能反映现实经济。在现实中，“看不见的手”的原理一般不能成立，帕累托最优状态通常也不能实现。这种情况就是“市场失灵”。\n\n市场失灵的情况包括不完全竞争、外部影响、公共物品、不完全信息等。'],
   ['pptx第6页',
    '市场失灵主要表现为：\n（一）收入和财富分配不公\n（二）竞争失败，形成垄断\n（三）失业问题\n（四）区域经济发展不协调\n（五）负外部性\n（六）公共物品'],
   ['pptx第7页', '二、外部性的定义'],
   ['pptx第8页',
    '（一）外部性概念的来源\n亨利·西奇威克：因为外部性因素的存在,会出现经济活动中个人成本、个人收益和社会成本、社会收益不一致的问题。\n\n阿尔弗雷德·马歇尔：当一个产业的产量扩大时,该产业各个企业的平均成本减少,这种情况就属于外部经济。扩大货物生产规模而发生的经济效率的提高可以分为“外部经济”和“内部经济”两类。\n\n庇古：在马歇尔的“外部经济”的基础上进一步提出了相对应的“外部不经济”概念。“外部不经济”和“外部经济”也被称为“负外部性”和“正外部性”。'],
   ['pptx第9页',
    '（二） 负外部性\n如果某个经济主体的行为使他人受损,却无须承担代价,就产生了负外部性。对损害的一方来说,不需要承担责任会导致他们失去减少这种行为的动力。在环境领域中,当某个经济主体从事某个活动时对自然环境造成不利影响,却没有把破坏环境的成本包含到产品和交易的成本中,就产生了环境负外部性。\n\n对于负外部性,经济主体行为的社会成本会大于其私人成本。假设一个

In [141]:
output = []

for pptx in pptx_data:
    file_name = pptx[0]
    content = pptx[1]
    content = json.dumps(content, ensure_ascii=False)

    ## get outline
    max_try_time = 5
    for i in range(max_try_time):
        try:
            outline = generate_outline(content)
            print(f'here is the outline: {outline}')
            outline = json.loads(outline)
            print("#############")
            print("success")
            print("#############")
            break
        except:
            print("#############")
            print(f"error: not json format: {file_name}, try again")
            print("#############")
            continue
    else:
        print("##########################")
        print(f"error: {file_name}, max try time reached")
        print("##########################")

    ## get summary
    summary = summarize(content)

    output.append([file_name, summary, outline])

    print([file_name, summary, outline])

here is the outline: [
  ["市场失灵理论", [4, 5, 6]],
  ["外部性理论", [7, 8, 9, 10, 11, 12]],
  ["碳排放的负外部性", [13, 14, 15, 16, 17]],
  ["全球公共物品的特殊性", [29, 30, 31, 32]],
  ["全球碳减排的囚徒困境", [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]],
  ["科斯理论", [48, 49, 50, 51, 52, 53, 54]],
  ["减缓碳排放的制度设计", [55, 56, 57, 58, 59, 60, 61, 62, 63]]
]
#############
success
#############
['第二章 《碳交易市场理论基础》--许光清.txt', '这份课件的核心内容可以总结为以下几点：\n\n1. **碳交易市场理论基础**：主要介绍了外部性理论、公共物品理论和全球碳减排的囚徒困境。\n\n2. **市场失灵**：阐述了完全竞争市场经济在现实中可能出现的失灵情况，包括不完全竞争、外部影响、公共物品、不完全信息等，并分析了市场失灵的主要表现。\n\n3. **外部性理论**：详细解释了外部性的定义、来源，并区分了正外部性和负外部性，特别强调了碳排放的负外部性及其全球性和长期性。\n\n4. **公共物品理论**：定义了公共物品和非排他性，并分析了公共物品如何导致市场失灵，以及碳排放的环境容量作为公共物品的特殊性。\n\n5. **全球碳减排的囚徒困境**：引入博弈论的概念，特别是囚徒困境模型，以说明全球碳减排合作中的困境，并提出了可能的缓解策略。\n\n6. **科斯理论**：介绍了科斯定理，包括交易费用理论、产权理论、契约理论以及企业理论等，并分析了科斯定理在碳交易市场中的应用。\n\n7. **减缓碳排放的制度设计**：讨论了减缓碳排放的制度设计目标、分类（行政手段、市场手段、信息手段）以及碳交易与碳税的原理，并对信息不完全情况下的碳税与碳排放进行了分析。\n\n8. **碳排放权交易制度与碳税制度的比较**：比较了两种制度在排放总量目标、减排成本、价格波动性、减排激励性和约束监管机制等方面的差异。\n\n9. **本章编写

In [142]:
type(output[0][2])

list

In [143]:
len(output)

9

In [144]:
## could discard this step is your outline do not need to trim the first and last element

def refine_output(output):
    for i in range(len(output)):
        outline = output[i][2]
        print(f'before trim{outline}')
        if len(outline[0][1])<=2:
            outline = outline[1:]
        if len(outline[-1][1])<=1:
            outline = outline[:-1]
        output[i][2] = outline
        print(f'after trim{outline}')

refine_output(output)
print(output)


"\ndef refine_output(output):\n    for i in range(len(output)):\n        outline = output[i][2]\n        print(f'before trim{outline}')\n        if len(outline[0][1])<=2:\n            outline = outline[1:]\n        if len(outline[-1][1])<=1:\n            outline = outline[:-1]\n        output[i][2] = outline\n        print(f'after trim{outline}')\n\nrefine_output(output)\nprint(output)\n"

In [145]:
## save the output
summary_outline_output_path = '/content/drive/My Drive/datasets/summary_outline'
for file_entry in output:
    file_name = file_entry[0]
    # Use json to save file to this folder
    with open(os.path.join(summary_outline_output_path, file_name), 'w') as f:
        json.dump(file_entry, f, ensure_ascii=False)

In [146]:
def load_data_from_files(folder_path):
    all_data = []
    for file_name in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file_name)
        # Ensure we are only opening JSON files
        if file_name.endswith('.txt'):
            with open(full_path, 'r', encoding='utf-8') as file:
                # Load the JSON content from the file
                data = json.load(file)
                all_data.append(data)
    return all_data

# Specify the path where the JSON files are stored
summary_outline_output_path = '/content/drive/My Drive/datasets/summary_outline'
summary_outline_data = load_data_from_files(summary_outline_output_path)




In [147]:
summary_outline_data[0]

['第二章 《碳交易市场理论基础》--许光清.txt',
 '这份课件的核心内容可以总结为以下几点：\n\n1. **碳交易市场理论基础**：主要介绍了外部性理论、公共物品理论和全球碳减排的囚徒困境。\n\n2. **市场失灵**：阐述了完全竞争市场经济在现实中可能出现的失灵情况，包括不完全竞争、外部影响、公共物品、不完全信息等，并分析了市场失灵的主要表现。\n\n3. **外部性理论**：详细解释了外部性的定义、来源，并区分了正外部性和负外部性，特别强调了碳排放的负外部性及其全球性和长期性。\n\n4. **公共物品理论**：定义了公共物品和非排他性，并分析了公共物品如何导致市场失灵，以及碳排放的环境容量作为公共物品的特殊性。\n\n5. **全球碳减排的囚徒困境**：引入博弈论的概念，特别是囚徒困境模型，以说明全球碳减排合作中的困境，并提出了可能的缓解策略。\n\n6. **科斯理论**：介绍了科斯定理，包括交易费用理论、产权理论、契约理论以及企业理论等，并分析了科斯定理在碳交易市场中的应用。\n\n7. **减缓碳排放的制度设计**：讨论了减缓碳排放的制度设计目标、分类（行政手段、市场手段、信息手段）以及碳交易与碳税的原理，并对信息不完全情况下的碳税与碳排放进行了分析。\n\n8. **碳排放权交易制度与碳税制度的比较**：比较了两种制度在排放总量目标、减排成本、价格波动性、减排激励性和约束监管机制等方面的差异。\n\n9. **本章编写人员**：说明了课件编写人员的背景和所属机构。\n\n这份课件旨在为理解碳交易市场的理论基础提供全面的指导，并对相关政策设计和执行提供理论支持。',
 [['市场失灵理论', [4, 5, 6]],
  ['外部性理论', [7, 8, 9, 10, 11, 12]],
  ['碳排放的负外部性', [13, 14, 15, 16, 17]],
  ['全球公共物品的特殊性', [29, 30, 31, 32]],
  ['全球碳减排的囚徒困境', [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]],
  ['科斯理论', [48, 49, 50, 51, 52, 53, 54]],
  ['减缓碳排放的制度设计', [55, 56, 57, 58, 59, 60

In [148]:
summary_outline_data[0][2]

[['市场失灵理论', [4, 5, 6]],
 ['外部性理论', [7, 8, 9, 10, 11, 12]],
 ['碳排放的负外部性', [13, 14, 15, 16, 17]],
 ['全球公共物品的特殊性', [29, 30, 31, 32]],
 ['全球碳减排的囚徒困境', [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]],
 ['科斯理论', [48, 49, 50, 51, 52, 53, 54]],
 ['减缓碳排放的制度设计', [55, 56, 57, 58, 59, 60, 61, 62, 63]]]

In [149]:
def write_helper(whole_outline, key_feature, ppt_content):
    system_role = "你是一名专业的教师，你擅长撰写高质量的教学资料，你正在根据大纲准备某一个具体知识点的讲义"
    user_message_part1 = "现在需要你大略阅读整个章节大纲: \n///"
    user_message_part2 = str(whole_outline)
    user_message_part3 = "///\n根据当前着重讲解的知识点：\n///"
    user_message_part4 = str(key_feature)
    user_message_part5 = "///\n参照下面的ppt内容：：\n///"
    user_message_part6 = str(ppt_content)
    user_message_part7 = "///\n  撰写该部分的讲义内容。要求内容详尽，易于理解"

    user_message = user_message_part1 + user_message_part2 + user_message_part3 + user_message_part4 + user_message_part5 + user_message_part6 + user_message_part7

    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_message}
    ]
    response = processor.process(messages)
    result = response.choices[0].message.content
    return result

In [150]:
def get_all_ppt_content(file_path = ppt_extracted_content_folder_path):
    ppt_content_dict = {}
    # Get a list of all files in the directory
    file_list = os.listdir(file_path)

    # Process each file in the filtered or full list
    for file_name in file_list:
        # Construct the full path to the file
        full_path = os.path.join(file_path, file_name)
        # Open and read the file, assuming it's in JSON format
        with open(full_path, 'r', encoding='utf-8') as file:
            # Load the content from JSON and add it to the dictionary
            content = json.load(file)
            ppt_content_dict[file_name] = content

    return ppt_content_dict

ppt_content_dict = get_all_ppt_content()



In [151]:
def prepare_ppt_content(index_list, file_name, ppt_content_dict = ppt_content_dict):
    ppt_content = ppt_content_dict[file_name]
    index_list = [x-1 for x in index_list]
    ppt_content = [ppt_content[i] for i in range(len(ppt_content)) if i in index_list]
    return ppt_content

result = prepare_ppt_content([1,2,4], "第二章 《碳交易市场理论基础》--许光清.txt")
print(result)

[['pptx第1页', ' 第二章 碳交易市场理论基础   高等学校碳金融“十四五”规划教材'], ['pptx第2页', ' 外部性理论 公共物品理论 全球碳减排的囚徒困境 目录CONTENTS 科斯理论'], ['pptx第4页', '一、市场失灵']]


In [152]:
def conbine_data(chapter_name, article, outline, summary, output_path = final_output_folder_path):

    title_str = chapter_name.split('.')[0]
    ## process article (type == list)
    article_str = "\n############################\n".join(article)


    ## process outline
    outline_str = "本章节重点内容总结及ppt中相对应的页码\n"
    for line in outline:
        title = line[0]
        indexes = line[1]
        indexes_str = ','.join([str(x) for x in indexes])
        outline_str += f'{title} -- 页码：{indexes_str}\n'

    ## process summary
    summary_str = summary

    result = f"""
{title_str}

{outline_str}
{summary_str}


################### 内容详解 ###################：
{article_str}
    """

    ## save into file
    with open(os.path.join(output_path, f'{title_str}.txt'), 'w', encoding='utf-8') as f:
        f.write(result)

    return result



In [153]:
summary

'该课件是关于高等学校碳金融“十四五”规划教材的一部分，主要内容包括：\n\n1. **碳关税形成的理论基础**：介绍了碳关税提出的背景、理论基础、文献综述、形成与发展等。\n   - 背景：全球气候变化、国际贸易规模的扩大、发展中国家的经济崛起等。\n   - 理论基础：外部性理论、碳泄露的环境冲突理论等。\n   - 文献综述：碳关税与关税、边境调节税的关系，碳关税的本质，福利效应等。\n   - 形成与发展：欧盟和美国提出碳关税计划，中国对碳关税的看法，其他主要经济体的回应。\n\n2. **欧盟碳边境调节机制的产生**：介绍了欧盟碳边境调节机制的产生背景、概念、主要内容等。\n   - 产生背景：欧盟委员会提出的一揽子政策提案，要求采取一系列改革措施以加速碳减排。\n   - 概念：碳边境调节机制是一种绿色贸易壁垒的新表现形式，是对欧盟碳排放权交易体系的补充。\n   - 主要内容：涉及产品、进口来源、实施时间和过渡期、含碳量计算、出口国的碳交易、含碳量报告和许可、碳边境调节机制证书的价格、实施方式、外国生产商的登记、罚则、反规避等。\n\n3. **欧盟碳边境调节机制的目标与路径**：介绍了欧盟碳边境调节机制的目标、路径简介、材料产品和温室气体清单、碳排放量计算方法等。\n   - 目标与路径：提出了六种路径方案，包括不同税种的路径分析方案和反映欧盟碳排放权交易体系法规的路径分析方案。\n   - 材料产品和温室气体清单：明确了碳边境调节机制涵盖的行业和排放物。\n   - 碳排放量计算方法：介绍了隐含碳排放量、简单产品、复杂产品等概念，以及如何确定特定产品的特定的隐含碳排放量。\n\n4. **欧盟碳边境调节机制影响分析**：分析了欧盟碳边境调节机制对欧盟和其他国家的影响。\n   - 对欧盟的影响：包括环境影响、宏观经济影响、重要行业影响、贸易影响、社会影响、收入影响等。\n   - 对其他国家的影响：分析了不同行业和国家可能受到的影响。\n\n5. **中国应对欧盟碳边境调节机制的对策**：提出了中国在应对欧盟碳边境调节机制问题上的立场、国际层面的对策、国内层面的措施等。\n   - 立场与原则：中国反对发达国家单方面实施碳边境调节措施，坚持共同但有区别的责任原则等。\n   - 国际层面的对策：通过环境外交等多种手段积极应对，积极参与国际相关标准的

In [154]:
print(conbine_data(file_name, article, outline, summary))



第九章  《碳关税与欧盟碳边境调节机制》--束兰根

本章节重点内容总结及ppt中相对应的页码
碳金融‘十四五’规划教材概述 -- 页码：1
碳关税形成的理论基础 -- 页码：4,5
欧盟碳边境调节机制的产生 -- 页码：8
欧盟碳边境调节机制的概念与主要内容 -- 页码：10,11,12,13,14,15
欧盟碳边境调节机制的原理与范围 -- 页码：16,17,18,19
欧盟碳边境调节机制的目标与路径 -- 页码：20,21,22,23,24,25,26,27
欧盟碳边境调节机制的材料、产品和温室气体清单 -- 页码：28
欧盟碳边境调节机制的碳排放量计算方法 -- 页码：29,30,31,32
欧盟碳边境调节机制的过渡性条款 -- 页码：33
欧盟碳边境调节机制影响分析 -- 页码：34,35,36,37,38,39,40
欧盟碳边境调节机制对其他国家的影响 -- 页码：41,42,43,44
碳边境调节机制的合法性分析 -- 页码：45,46
中国应对欧盟碳边境调节机制的对策 -- 页码：47,48,49,50

该课件是关于高等学校碳金融“十四五”规划教材的一部分，主要内容包括：

1. **碳关税形成的理论基础**：介绍了碳关税提出的背景、理论基础、文献综述、形成与发展等。
   - 背景：全球气候变化、国际贸易规模的扩大、发展中国家的经济崛起等。
   - 理论基础：外部性理论、碳泄露的环境冲突理论等。
   - 文献综述：碳关税与关税、边境调节税的关系，碳关税的本质，福利效应等。
   - 形成与发展：欧盟和美国提出碳关税计划，中国对碳关税的看法，其他主要经济体的回应。

2. **欧盟碳边境调节机制的产生**：介绍了欧盟碳边境调节机制的产生背景、概念、主要内容等。
   - 产生背景：欧盟委员会提出的一揽子政策提案，要求采取一系列改革措施以加速碳减排。
   - 概念：碳边境调节机制是一种绿色贸易壁垒的新表现形式，是对欧盟碳排放权交易体系的补充。
   - 主要内容：涉及产品、进口来源、实施时间和过渡期、含碳量计算、出口国的碳交易、含碳量报告和许可、碳边境调节机制证书的价格、实施方式、外国生产商的登记、罚则、反规避等。

3. **欧盟碳边境调节机制的目标与路径**：介绍了欧盟碳边境调节机制的目标、路径简介、材料产品和温室气体清单、碳排放量计算方

In [155]:
for data in summary_outline_data:
    file_name = data[0]
    summary = data[1]
    outline = data[2]
    article = []
    try:
        for part in outline:
            title = part[0]
            index_list = part[1]
            ppt_content = prepare_ppt_content(index_list, file_name)
            ppt_content = str(ppt_content)

            article_piece = write_helper(whole_outline = outline, key_feature = title, ppt_content = ppt_content)
            """
            print("########################")
            print(f'index list: {index_list}')
            print(f'title: {title}')
            print(f'ppt content: {ppt_content}')
            print(f'article: {article}')
            print("########################")
            """
            article.append(article_piece)
        conbine_data(file_name, article, outline, summary)
    except:
        print(f'error: {file_name}')
        continue

    print('############# raw article ')
    print(article)
    print('############# refined article ')
    print(refined_article)
    break

############# raw article 
['讲义内容：市场失灵理论\n\n一、引言\n\n在经济学中，市场机制被认为是资源配置的有效方式。完全竞争市场经济在一系列理想化假定条件下，可以导致整个经济达到一般均衡，实现资源配置的帕累托最优状态。然而，现实中的市场并非完全符合理想化的假定条件，因此市场机制有时并不能实现预期的效果，导致“市场失灵”。本讲义将详细阐述市场失灵的理论及其主要表现。\n\n二、市场失灵的原因\n\n1. 完全竞争市场假定条件的严格性\n\n完全竞争市场假定条件包括：市场上有很多买家和卖家，产品同质化，信息完全对称，不存在交易成本等。这些假定条件在现实中很难同时满足，导致完全竞争市场和其他一系列理想化假定条件并不能反映现实经济。\n\n2. 市场失灵的表现形式\n\n市场失灵主要表现在以下几个方面：\n\n（1）不完全竞争\n\n不完全竞争是指市场上存在垄断、寡头垄断等市场结构，使得市场机制无法实现资源配置的帕累托最优。在这种情况下，垄断企业可以操纵市场价格，损害消费者利益。\n\n（2）外部影响\n\n外部影响是指市场交易双方在交易过程中对第三方产生的影响，包括正外部性和负外部性。正外部性如科技创新、绿化等，负外部性如污染、噪音等。市场机制无法内部化外部影响，导致资源配置失衡。\n\n（3）公共物品\n\n公共物品具有非竞争性和非排他性，市场机制无法有效提供公共物品。例如，国防、路灯等公共设施，市场无法确保其有效供应。\n\n（4）不完全信息\n\n信息不完全是指市场上买卖双方在交易过程中所掌握的信息不对称。在这种情况下，市场机制无法实现资源配置的帕累托最优。例如，消费者无法完全了解商品的质量，导致市场出现逆向选择现象。\n\n三、市场失灵的主要表现\n\n以下是市场失灵的几种主要表现：\n\n1. 收入和财富分配不公\n\n市场机制在资源配置过程中，可能导致收入和财富分配不公。这主要是因为市场竞争的结果往往使强者更强，弱者更弱，从而加剧社会贫富差距。\n\n2. 竞争失败，形成垄断\n\n市场机制无法完全消除垄断现象，垄断企业可以通过操纵市场价格和行业标准来损害消费者利益，导致市场失灵。\n\n3. 失业问题\n\n市场机制在调节劳动力市场时，可能无法实现充分就业。失业问题不仅影响个人生活，还会导致社会不稳定。\n\n4. 区域经